pip install --upgrade pandas pymysql

In [5]:
import pymysql.cursors
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

import json
import time

import numpy as np

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver

from bs4 import BeautifulSoup as BS
import bs4
import soupsieve

import pymysql
from typing import List, Dict, Optional
from pymysql.cursors import DictCursor

import warnings
warnings.filterwarnings('ignore')

In [6]:
df_urls = pd.read_excel('C:/Users/Personnel/Ligue 1 Project/ws_url_teams.xlsx')
df_urls

,url,league,id,team
0,https://fr.whoscored.com/Teams/304/Fixtures/Fr...,France-Ligue-1,304,Paris-Saint-Germain
1,https://fr.whoscored.com/Teams/309/Fixtures/Fr...,France-Ligue-1,309,Lens
2,https://fr.whoscored.com/Teams/613/Fixtures/Fr...,France-Ligue-1,613,Nice
3,https://fr.whoscored.com/Teams/248/Fixtures/Fr...,France-Ligue-1,248,Monaco
4,https://fr.whoscored.com/Teams/607/Fixtures/Fr...,France-Ligue-1,607,Lille
...,...,...,...,...
91,https://www.whoscored.com/Teams/189/Fixtures/A...,Angleterre-Premier-League,189,Brentford
92,https://www.whoscored.com/Teams/170/Fixtures/A...,Angleterre-Premier-League,170,Fulham
93,https://www.whoscored.com/Teams/184/Fixtures/A...,Angleterre-Premier-League,184,Burnley
94,https://www.whoscored.com/Teams/163/Fixtures/A...,Angleterre-Premier-League,163,Sheffield-United


In [10]:
# Define the leagues you're interested in
leagues = ['Allemagne-Bundesliga', 'England-Premier-League', 'Espagne-LaLiga', 'France-Ligue-1', 'Italie-Serie-A']

# Initialize an empty set to store unique URLs (sets inherently avoid duplicates)
all_urls = set()

# Initialize the WebDriver
driver = webdriver.Chrome()  # or webdriver.Firefox(), etc., depending on your browser and setup

# Iterate through each row in the DataFrame
for index, row in df_urls.iterrows():
    # Extract the URL and league from the 'url' column
    team_fixtures_url = row['url']
    
    # Use the WebDriver to visit the page
    driver.get(team_fixtures_url)
    
    # Add a delay to avoid overloading the server and to ensure the page loads
    time.sleep(2)
    
    # Use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract additional URLs from the page
    # Filter out the links that have '/Live/' and belong to the specified leagues
    additional_urls = []
    for league in leagues:
        additional_urls.extend(soup.select(f'a[href*="/Live/"][href*="{league.replace(" ", "-")}"]'))
    
    # Process each found URL
    for link in additional_urls:
        href = link.get('href')
        if href:  # Make sure href is not None
            full_url = f'https://www.whoscored.com{href}'
            all_urls.add(full_url)  # Set ensures only unique URLs are added

# Convert the set back to a list if needed
unique_urls_list = list(all_urls)

# Print or use the list of all unique URLs
for url in unique_urls_list:
    print(url)

# Make sure to close the WebDriver after the scraping job is done
driver.quit()


https://www.whoscored.com/Matches/1746131/Live/Italie-Serie-A-2023-2024-Genoa-Salernitana
https://www.whoscored.com/Matches/1734670/Live/Espagne-LaLiga-2023-2024-Osasuna-Barcelona
https://www.whoscored.com/Matches/1729256/Live/England-Premier-League-2023-2024-Arsenal-Fulham
https://www.whoscored.com/Matches/1743485/Live/Allemagne-Bundesliga-2023-2024-Wolfsburg-Bayern-Munich
https://www.whoscored.com/Matches/1734639/Live/Espagne-LaLiga-2023-2024-Girona-Getafe
https://www.whoscored.com/Matches/1743457/Live/Allemagne-Bundesliga-2023-2024-Wolfsburg-Bayer-Leverkusen
https://www.whoscored.com/Matches/1741077/Live/France-Ligue-1-2023-2024-Paris-Saint-Germain-Lorient
https://www.whoscored.com/Matches/1734631/Live/Espagne-LaLiga-2023-2024-Almeria-Real-Madrid
https://www.whoscored.com/Matches/1734899/Live/Espagne-LaLiga-2023-2024-Barcelona-Osasuna
https://www.whoscored.com/Matches/1734767/Live/Espagne-LaLiga-2023-2024-Sevilla-Getafe
https://www.whoscored.com/Matches/1734629/Live/Espagne-LaLiga-2

In [13]:
print(len(all_urls))

1091


In [14]:
def extract_match_info(url):
    # Extract match_id after 'Matches/'
    match_id_start = url.find('Matches/') + len('Matches/')
    match_id_end = url.find('/Live/')
    match_id = url[match_id_start:match_id_end]
    
    # Extract league after 'Live/'
    league_start = url.find('/Live/') + len('/Live/')
    league_end = url.find('-2023-2024')
    league = url[league_start:league_end]
    
    # Extract teams after '2024-'
    teams_start = url.find('2024-') + len('2024-')
    teams = url[teams_start:]
    
    return match_id, league, teams


connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)

for url in unique_urls_list:
    match_id, league, teams = extract_match_info(url)
    
    # Insert the extracted data into the MySQL table
    with connection.cursor() as cursor:
        insert_query = "INSERT IGNORE INTO matchs (match_id, league, teams, match_link) VALUES (%s, %s, %s, %s)"
        data = (match_id, league, teams, url)
        cursor.execute(insert_query, data)
        connection.commit()

# Close the connection
connection.close()


In [15]:
class MatchEvent(BaseModel):
    id: int
    match_id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    player_id: int
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [16]:
class Player(BaseModel):
    player_id: int
    shirt_no: int
    name: str
    age: int
    position: str
    team_id: int

In [30]:
matchs_scraped = df_events['match_id'].unique()
matchs_scraped

array([1729191, 1729190, 1729192, 1729195, 1741060, 1734629, 1734626,
       1741077, 1734622, 1741062, 1741064, 1729197, 1734630, 1729199,
       1734623, 1743393, 1743397, 1743396, 1729254, 1746042, 1741221,
       1734631, 1746043, 1746047, 1741217, 1729252, 1741224, 1729250,
       1743395, 1746045, 1746044, 1734639, 1734632, 1741223, 1734637,
       1746046, 1746040, 1729203, 1743400, 1741233, 1729260, 1734642,
       1729255, 1743407, 1743405, 1743408, 1743401, 1729256, 1729262,
       1729257, 1743404, 1746058, 1734645, 1734643, 1746053, 1746050,
       1741226, 1734648, 1741228, 1741234, 1743403, 1729263, 1746052,
       1741231, 1746055, 1734650, 1746051, 1746067, 1734662, 1743409,
       1746069, 1729272, 1729274, 1734674, 1743416, 1743411, 1743417,
       1729268, 1729266, 1734672, 1741243, 1734666, 1743413, 1729267,
       1746061, 1734664, 1741241, 1741237, 1729270, 1734668, 1741238,
       1746062, 1746063, 1734670, 1741244, 1734695, 1729284, 1734676,
       1743421, 1743

In [25]:
def is_match_id_scraped(match_id, connection):
    with connection.cursor() as cursor:
        query = "SELECT EXISTS(SELECT 1 FROM match_events WHERE match_id = %s)"
        cursor.execute(query, (match_id,))
        result = cursor.fetchone()
        # Check if the result is a dictionary and get the value appropriately
        if isinstance(result, dict):
            return result['EXISTS(SELECT 1 FROM match_events WHERE match_id = %s)']
        else:
            # Assume the result is a tuple and return the first element
            return result[0]

def scrape_match_events(whoscored_url, driver, connection):
    
    match_id = int(whoscored_url.split('/Matches/')[-1].split('/Live/')[0])
    
    if is_match_id_scraped(match_id, connection):
        print(f"Match with ID {match_id} has already been scraped.")
        return  # Skip this match
    
    driver.get(whoscored_url)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    element = soup.select_one('script:-soup-contains("matchCentreData")')
    
    matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(',\n')[0])
    
    match_events = matchdict['events']
    
    df = pd.DataFrame(match_events)
    
    df.dropna(subset='playerId', inplace=True)
    
    df = df.where(pd.notnull(df), None)
    
    df['match_id'] = match_id
    
    if 'match_id' not in df.columns:
        raise KeyError("Column 'match_id' missing from DataFrame")
    
    df = df.rename(
    {
        'eventId': 'event_id',
        'expandedMinute': 'expanded_minute',
        'outcomeType': 'outcome_type',
        'isTouch': 'is_touch',
        'playerId': 'player_id',
        'teamId': 'team_id',
        'endX': 'end_x',
        'endY': 'end_y',
        'blockedX': 'blocked_x',
        'blockedY': 'blocked_y',
        'goalMouthZ': 'goal_mouth_z',
        'goalMouthY': 'goal_mouth_y',
        'isShot': 'is_shot',
        'cardType': 'card_type',
        'isGoal': 'is_goal'
    },
        axis=1
    )
    
    df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
    df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
    df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])
    
    df.drop(columns=["period", "type", "outcome_type"], inplace=True)
    
    if 'is_goal' not in df.columns:
        df['is_goal'] = False
        
    if 'is_card' not in df.columns:
        df['is_card'] = False
        df['card_type'] = False
        
    df = df[~(df['type_display_name'] == "OffsideGiven")]
    
    df = df[[
        'id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
        'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
        'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
        'period_display_name','match_id'
    ]]
    
    df[['id', 'event_id', 'minute', 'team_id', 'player_id','match_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id','match_id']].astype(np.int64)
    df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)
    df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)
    
    df['is_goal'] = df['is_goal'].fillna(False)
    df['is_shot'] = df['is_shot'].fillna(False)
    
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.float32:
            df[column] = np.where(
                np.isnan(df[column]),
                None,
                df[column]
            )
                
    
    team_info = []
    team_info.append({
        'team_id': matchdict['home']['teamId'],
        'name': matchdict['home']['name'],
        'country_name': matchdict['home']['countryName'],
        'manager_name': matchdict['home']['managerName'],
        'players': matchdict['home']['players'],
    })

    team_info.append({
        'team_id': matchdict['away']['teamId'],
        'name': matchdict['away']['name'],
        'country_name': matchdict['away']['countryName'],
        'manager_name': matchdict['away']['managerName'],
        'players': matchdict['away']['players'],
    })
    
    #print(df.head())
    
    insert_players(team_info, connection)
    insert_match_events(df, connection)
    
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)    


In [26]:
def insert_players(team_info, connection):
    insert_query = """
    INSERT INTO players (player_id, name, shirt_no, age, position, team_id)
    VALUES (%(player_id)s, %(name)s, %(shirt_no)s, %(age)s, %(position)s, %(team_id)s)
    ON DUPLICATE KEY UPDATE name = VALUES(name), shirt_no = VALUES(shirt_no), age = VALUES(age), position = VALUES(position);
    """
    with connection.cursor() as cursor:
        for team in team_info:
            for player in team['players']:
                # Create a dictionary for the player
                player_data = {
                    'player_id': player['playerId'],
                    'name': player['name'],
                    'shirt_no': player.get('shirtNo', None),  # Using .get() to handle missing data
                    'age': player.get('age', None),
                    'position': player.get('position', None),
                    'team_id': team['team_id']
                }
                cursor.execute(insert_query, player_data)
        connection.commit()


def insert_match_events(df, connection):
    with connection.cursor() as cursor:
        for _, row in df.iterrows():
            insert_query = """
                INSERT INTO match_events 
                (id, match_id, event_id, minute, second, team_id, player_id, x, y, end_x, end_y, qualifiers, is_touch, blocked_x, blocked_y, goal_mouth_z, goal_mouth_y, is_shot, card_type, is_goal, type_display_name, outcome_type_display_name, period_display_name) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                minute = VALUES(minute), second = VALUES(second), x = VALUES(x), y = VALUES(y), end_x = VALUES(end_x), end_y = VALUES(end_y), is_shot = VALUES(is_shot), is_goal = VALUES(is_goal)
            """
            # Prepare your data tuple from the row here
            data = (row['id'], row['match_id'], row['event_id'], row['minute'], row['second'], row['team_id'], row['player_id'], row['x'], row['y'], row['end_x'], row['end_y'], json.dumps(row['qualifiers']), row['is_touch'], row['blocked_x'], row['blocked_y'], row['goal_mouth_z'], row['goal_mouth_y'], row['is_shot'], row['card_type'], row['is_goal'], row['type_display_name'], row['outcome_type_display_name'], row['period_display_name'])
            
            cursor.execute(insert_query, data)
        connection.commit()


In [27]:
import warnings
warnings.filterwarnings('ignore')


driver = webdriver.Chrome()

for i,url in enumerate(all_urls,start=1):
    print(f"{i}- {url}")
    scrape_match_events(
        whoscored_url=url,
        driver=driver, 
        connection=connection)
    
    time.sleep(2)

1- https://www.whoscored.com/Matches/1746131/Live/Italie-Serie-A-2023-2024-Genoa-Salernitana


KeyError: 'EXISTS(SELECT 1 FROM match_events WHERE match_id = %s)'

In [20]:
# Establish a connection to the MySQL database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM match_events"

# Use pandas to load the query result into a DataFrame
df_events = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_events

,id,match_id,event_id,minute,second,team_id,player_id,x,y,end_x,...,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
0,2575436797,1729191,3,0,0.0,184,371447,50.0,50.0,27.2,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
1,2575436839,1729191,3,0,5.0,167,122945,27.8,73.4,NaN,...,NaN,NaN,NaN,NaN,0,0,0,Aerial,Successful,FirstHalf
2,2575436867,1729191,4,0,2.0,184,353412,28.7,59.3,65.1,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,FirstHalf
3,2575436873,1729191,5,0,5.0,184,419917,72.2,26.6,NaN,...,NaN,NaN,NaN,NaN,0,0,0,Aerial,Unsuccessful,FirstHalf
4,2575436877,1729191,4,0,6.0,167,122945,23.9,77.7,40.7,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,FirstHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858501,2645915661,1729472,648,94,25.0,162,231135,29.1,40.1,35.9,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,SecondHalf
858502,2645915711,1729472,649,94,29.0,162,389579,35.9,72.0,96.0,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,SecondHalf
858503,2645915997,1729472,1241,95,36.0,15,395692,94.0,4.6,NaN,...,NaN,NaN,NaN,NaN,0,0,0,TakeOn,Successful,SecondHalf
858504,2645916003,1729472,650,95,36.0,162,409440,6.0,95.4,NaN,...,NaN,NaN,NaN,NaN,0,0,0,Challenge,Unsuccessful,SecondHalf


In [21]:
import pymysql

# Establish a connection to the MySQL database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM players"

# Use pandas to load the query result into a DataFrame
df_players = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_players

,player_id,shirt_no,name,age,position,team_id
0,2987,1,Pepe Reina,41,Sub,839
1,4511,6,James Milner,38,DL,211
2,6227,14,Dani Rodríguez,35,Sub,51
3,6683,4,Dante,40,DC,613
4,8105,31,Andy Lonergan,40,Sub,31
...,...,...,...,...,...,...
2942,515865,62,Joshua King,17,Sub,170
2943,515888,41,Junior Ndiaye,18,Sub,311
2944,515943,65,Pietro Passador,20,Sub,72
2945,516051,41,Stanis Idumbo Muzambo,18,Sub,67


In [134]:
import pymysql

# Establish a connection to the MySQL database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM players"

# Use pandas to load the query result into a DataFrame
df_players = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_players

C:\Users\Personnel\AppData\Local\Temp\ipykernel_10108\3083184940.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_players = pd.read_sql(query, connection)


,player_id,shirt_no,name,age,position,team_id
0,46092,1,Martin Dúbravka,35,Sub,23
1,69344,14,Christian Eriksen,31,Sub,32
2,73063,15,Craig Dawson,33,DC,161
3,74341,32,Joël Matip,32,DC,26
4,82277,33,Dan Burn,31,DL,23
...,...,...,...,...,...,...
75,430350,50,Ben Doak,18,Sub,26
76,431283,24,Toti,25,Sub,161
77,437022,49,Alejandro Garnacho,19,ML,32
78,443578,32,Joe Hodge,21,Sub,161


In [135]:
# Establish a connection to the MySQL database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM match_events"

# Use pandas to load the query result into a DataFrame
df_events = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_events

C:\Users\Personnel\AppData\Local\Temp\ipykernel_10108\1154523515.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_events = pd.read_sql(query, connection)


,id,match_id,event_id,minute,second,team_id,player_id,x,y,end_x,...,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
0,2577367867,1729199,3,0,0.0,32,123761,49.9,49.9,37.0,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
1,2577367875,1729199,4,0,2.0,32,88526,37.0,50.1,35.7,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
2,2577367885,1729199,5,0,3.0,32,118244,35.7,83.6,29.7,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
3,2577367891,1729199,6,0,5.0,32,340105,29.7,67.0,35.4,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
4,2577367899,1729199,7,0,7.0,32,88526,35.4,59.7,24.6,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3279,2583294891,1729263,782,93,59.0,26,136451,15.9,47.3,21.1,...,NaN,NaN,NaN,NaN,0,0,0,Clearance,Successful,SecondHalf
3280,2583456607,1729263,1295,26,21.0,26,114147,19.1,45.8,30.1,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
3281,2583456659,1729263,1297,26,23.0,26,86829,26.5,41.7,23.2,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
3282,2583456735,1729263,1299,26,25.0,26,74341,23.4,22.1,10.0,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf


In [55]:
try:
    with connection.cursor() as cursor:
        cursor.execute("SELECT VERSION()")
        version = cursor.fetchone()
        print("Database version:", version)
except pymysql.Error as e:
    print("Database error:", e)
    # Reconnect or handle error appropriately


Database error: (0, '')
